In [1]:
import os 

import pandas as pd
import numpy as np
import scanpy as sc


from SURE import DensityFlow
from SURE.flow import plot_quiver
from SURE.flow import VectorFieldEval
from SURE.perturb import LabelMatrix, DoseMatrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from eval_metrics import mmd_eval, r2_score_eval, pearson_eval

import torch
torch.set_float32_matmul_precision("high")

import seaborn as sns
import matplotlib.pyplot as plt

os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 仅显示GPU 0


import re

def remove_g_suffix(text):
    """
    移除以 'g' + 数字结尾的部分（如 "abcg123" → "abc"）
    """
    return re.sub(r'g\d+$', '', text)

np.random.seed(42) 

pert_col = 'cytokine'
control_label = 'PBS'
loss_func = 'negbinomial'

In [2]:
adata_ = sc.read_h5ad('donor4.h5ad')
sc.pp.filter_genes(adata_, min_cells=10)
sc.pp.normalize_total(adata_)
sc.pp.log1p(adata_)
adata = adata_.copy()
adata.shape

(578242, 1977)

## Split data into two subsets for train and test

In [3]:
cells_pert = adata[adata.obs[pert_col]!=control_label].obs_names
cells_train, cells_test = train_test_split(cells_pert, test_size= adata.shape[0] // 8)
cells_train = cells_train.tolist() + adata[adata.obs[pert_col]==control_label].obs_names.tolist()
adata_train = adata[cells_train].copy()
adata_test = adata[cells_test].copy()

In [4]:
xs = adata_train.X

lb1 = LabelMatrix()
us1 = lb1.fit_transform(adata_train.obs[pert_col],control_label)
ln1 = lb1.labels_
zb1 = [True] * us1.shape[1]

us = us1 
ln = ln1 
zb = zb1 
us.shape


(505962, 90)

## Train the model

In [5]:
model = DensityFlow(input_size = xs.shape[1],
                      cell_factor_size=us.shape[1],
                      codebook_size=100,
                      z_dim=50,
                      hidden_layers=[1024],
                      z_dist='studentt',
                      loss_func=loss_func,
                      use_zeroinflate=False,
                      seed=42,
                      zero_bias=zb,
                      use_cuda=True)

🧬 DensityFlow Initialized:
   - Codebook size: 100
   - Latent Dimension: 50
   - Gene Dimension: 1977
   - Hidden Dimensions: [1024]
   - Device: cuda:0
   - Parameters: 14,164,601


In [6]:
%%time 

model.fit(xs, 
          us=us, 
          num_epochs=200, 
          batch_size=1000, 
          use_jax=True)

Training: 100%|██████████| 200/200 [5:23:44<00:00, 97.12s/epoch, loss=281346.3081]  

CPU times: user 5h 24min 1s, sys: 7.85 s, total: 5h 24min 9s
Wall time: 5h 23min 48s


In [7]:
DensityFlow.save_model(model, f'densityflow_{loss_func}_model.pt')

Model saved to /home/oem/Workspace/PerturbFlow_Ex/Parse10M/densityflow_negbinomial_model.pt


In [8]:
model = DensityFlow.load_model(f'densityflow_{loss_func}_model.pt')

Model loaded from densityflow_negbinomial_model.pt
🧬 DensityFlow Initialized:
   - Codebook size: 100
   - Latent Dimension: 50
   - Gene Dimension: 1977
   - Hidden Dimensions: [1024]
   - Device: cuda:0
   - Parameters: 14,164,601


## Prediction for test data

In [9]:
adata_test = adata_test[adata_test.obs[pert_col].isin(ln)]
#xs_test = adata_test.X.toarray()
#zs_test_basal = model.get_basal_embedding(xs_test)

In [10]:
def predict_pert_effect(ad,pert):
    ad = ad.copy()
    xs_pert = ad.X.toarray()
    zs_basal = model.get_basal_embedding(xs_pert)
    us_pert = np.ones([xs_pert.shape[0],1])

    ind = int(np.where(ln==pert)[0])
    dzs = model.get_cell_shift(zs_basal, perturb_idx=ind, perturb_us=us_pert)
    
    library_sizes = np.sum(xs_pert, axis=1, keepdims=True)
    counts = model.get_counts(zs_basal+dzs, library_sizes=library_sizes)
    return counts.copy()

def normalize(counts):
    ad = sc.AnnData(counts)
    sc.pp.normalize_total(ad)
    sc.pp.log1p(ad)
    return ad.X

## Evaluation

In [11]:
adata_control = adata_train[adata_train.obs[pert_col]==control_label].copy()

In [12]:
results = []
ad_results = []
pert_sets = adata_test.obs[pert_col].unique().tolist()
i = 0
for pert in pert_sets:
    i += 1
    print(f'{i}/{len(pert_sets)}')
    
    ad_test = adata_test[adata_test.obs[pert_col]==pert].copy()
    xs_test = ad_test.X.toarray()
    
    ind = np.random.choice(np.arange(adata_control.shape[0]), size=ad_test.shape[0], replace=True)
    ad_ctrl = adata_control[ind].copy()
    ad_ctrl.obs_names_make_unique()
    xs_basal = ad_ctrl.X.toarray()
    
    xs_test_pred = predict_pert_effect(ad_test, pert)
    #xs_test_pred = normalize(xs_test_pred_)
    #xs_test = normalize(xs_test)
    #xs_basal = normalize(xs_basal)
    
    mmd_value=mmd_eval(xs_test_pred, xs_test)
    r2 = r2_score_eval(xs_test_pred, xs_test)
    pr = pearson_eval(xs_test_pred-xs_basal,xs_test-xs_basal)
    print(f'mmd:{mmd_value}; r2:{r2}; pearson:{pr}')
    results.append({'mmd':mmd_value,'r2':r2,'pearson':pr})
    
    ad = sc.AnnData(xs_test_pred)
    ad.var_names = adata.var_names
    ad.obs_names_make_unique()
    ad.obs['target_gene'] = pert
    ad_results.append(ad)


1/90


100%|██████████| 1/1 [00:00<00:00, 148.53batch/s]


mmd:0.0011940213153138757; r2:0.9948893189430237; pearson:0.8619739413261414
2/90


100%|██████████| 1/1 [00:00<00:00, 92.15batch/s]


mmd:0.000814066908787936; r2:0.9962961077690125; pearson:0.9523965716362
3/90


...

mmd:0.000908108486328274; r2:0.9964379668235779; pearson:0.8563327789306641
89/90



100%|██████████| 1/1 [00:00<00:00, 183.77batch/s]


mmd:0.0009940736927092075; r2:0.9941883683204651; pearson:0.9857034683227539
90/90


100%|██████████| 1/1 [00:00<00:00, 151.62batch/s]


mmd:0.000961520301643759; r2:0.9955268502235413; pearson:0.8461726307868958


In [13]:
df = pd.DataFrame(results)
df.head()

,mmd,r2,pearson
0,0.001194,0.994889,0.861974
1,0.000814,0.996296,0.952397
2,0.000997,0.996372,0.711416
3,0.000858,0.994811,0.790988
4,0.000758,0.995899,0.989949


In [14]:
df.mean(axis=0)

mmd        0.001035
r2         0.995982
pearson    0.885209
dtype: float64

In [15]:
df.to_csv(f'benchmark_{loss_func}_densityflow.csv')

In [16]:
adata_pred = sc.concat(ad_results)
adata_pred.obs_names_make_unique()

In [17]:
adata_pred.X.max()

23.874306

In [18]:
adata_control.obs['target_gene'] = 'non-targeting'

In [19]:
adata_pred = sc.concat([adata_pred, adata_control])
adata_pred.obs_names_make_unique()

adata_pred.write('df_pred.h5ad')

In [20]:
adata_pred.var_names.to_series().to_csv('gene_name.csv', header=None, index=None)

In [21]:
adata_test.obs['target_gene'] = adata_test.obs[pert_col].copy()
adata_test = sc.concat([adata_test, adata_control])

adata_test.write('test.h5ad')

%%bash

cell-eval baseline -a test.h5ad 
cell-eval run -ap baseline.h5ad -ar test.h5ad --num-threads 8 -o cell-eval-baseline-outdir --profile vcc

In [22]:
%%bash 

cell-eval run -ap df_pred.h5ad -ar test.h5ad --num-threads 8 --profile vcc

INFO:cell_eval._evaluator:Reading real anndata from test.h5ad
INFO:cell_eval._evaluator:Reading pred anndata from df_pred.h5ad
INFO:cell_eval.utils:Data appears to be log1p normalized (decimals detected, range [0.00, 5.42])
INFO:cell_eval._evaluator:Input is found to be log-normalized already - skipping transformation.
Traceback (most recent call last):
  File "/home/oem/anaconda3/envs/SUREv1/bin/cell-eval", line 7, in <module>
    sys.exit(main())
  File "/home/oem/anaconda3/envs/SUREv1/lib/python3.10/site-packages/cell_eval/__main__.py", line 37, in main
    run_evaluation(args)
  File "/home/oem/anaconda3/envs/SUREv1/lib/python3.10/site-packages/cell_eval/_cli/_run.py", line 185, in run_evaluation
    evaluator = MetricsEvaluator(
  File "/home/oem/anaconda3/envs/SUREv1/lib/python3.10/site-packages/cell_eval/_evaluator.py", line 84, in __init__
    self.anndata_pair = _build_anndata_pair(
  File "/home/oem/anaconda3/envs/SUREv1/lib/python3.10/site-packages/cell_eval/_evaluator.py", 

CalledProcessError: Command 'b'\ncell-eval run -ap df_pred.h5ad -ar test.h5ad --num-threads 8 --profile vcc\n'' returned non-zero exit status 1.

In [ ]:
%%bash 
cell-eval score -i cell-eval-outdir/agg_results.csv -I cell-eval-baseline-outdir/agg_results.csv -o cell-eval-outdir/overall_results.csv

INFO:cell_eval._score:Reading user results from cell-eval-outdir/agg_results.csv
INFO:cell_eval._score:Reading base results from cell-eval-baseline-outdir/agg_results.csv
INFO:cell_eval._score:user score: [0.09370245 0.02418927 0.7792    ]
INFO:cell_eval._score:base score: [0.01629919 0.02717401 0.52      ]
INFO:cell_eval._score:Calculating norm by zero for 1 metrics: mae
INFO:cell_eval._score:Calculating norm by one for 2 metrics: overlap_at_N, discrimination_score_l1
INFO:cell_eval._score:Writing results to cell-eval-outdir/overall_results.csv
